# Advanced processing of Strands Agents Response

Strands Agents allows you to intercept and process events as they happen during agent execution using two methods: 
    
- **Async iterators**: ideal for asynchronous frameworks like FastAPI, aiohttp, or Django Channels. For these environments, the SDK offers the `stream_async` method which returns an asynchronous iterator. 
- **Callback handlers**: allow you to intercept and process events as they happen during agent execution. This enables real-time monitoring, custom output formatting, and integration with external systems.

In this example, we will show you how to use both methods to handle calls on your agent


## Agent Details
<div style="float: left; margin-right: 20px; ">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |async iterators, callback handlers                 |
|Agent Structure     |single agent architecture                          |
|Native tools used   |calculator                                         |
|Custom tools created|Weather forecast                                   |

</div>

## Architecture

<div style="text-align:left;">
    <img src="./images/architecture.png" width="65%" />
</div>

## Key Features
* Async Iterators for Streaming
* Callback Handlers


## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent Agent

In [ ]:
# installing pre-requisites
# !pip install -r requirements.txt

### Importing dependency packages

Now let's import the dependency packages

In [1]:
import asyncio

import httpx
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
from strands import Agent, tool
from strands_tools import calculator

## Method 1 - Async Iterators for Streaming


Strands Agents provides support for asynchronous iterators through the `stream_async` method, enabling real-time streaming of agent responses in asynchronous environments like web servers, APIs, and other async applications.

Since we are show casing this example in a notebook, we need to apply `nest_asyncio` to allow nested use of `asyncio.run` and `loop.run_until_complete`

In [2]:
nest_asyncio.apply()

### Creating and invoking agent with stream_async

Let's now create our agent with a built-in calculator tool and no `callback_handler`. We will use the `stream_async` method to iteract over the streamed agent events

In [ ]:
# Initialize our agent without a callback handler
from strands.models import BedrockModel

# --- Create the Bedrock model using your inference profile ARN ---
   

# inference_profile_arn="arn:aws:bedrock:ap-southeast-2:381491838394:inference-profile/au.anthropic.claude-haiku-4-5-20251001-v1:0",
    
agent = Agent(
    # model= "anthropic.claude-haiku-4-5-20251001-v1:0"
    # model="anthropic.claude-3-5-sonnet-20241022-v2:0",  # Optional: Specify the model ID
    model= model,
    tools=[calculator], 
    callback_handler=None)

# Async function that iterators over streamed agent events


async def process_streaming_response():
    agent_stream = agent.stream_async("Calculate 2+2")
    async for event in agent_stream:
        print(event)


# Run the agent
asyncio.run(process_streaming_response())

/tmp/ipykernel_338769/3892389870.py:5: UserWarning: Invalid configuration parameters: ['inference_profile_arn', 'provider', 'region'].
Valid parameters are: ['additional_args', 'additional_request_fields', 'additional_response_field_paths', 'cache_prompt', 'cache_tools', 'guardrail_id', 'guardrail_redact_input', 'guardrail_redact_input_message', 'guardrail_redact_output', 'guardrail_redact_output_message', 'guardrail_stream_processing_mode', 'guardrail_trace', 'guardrail_version', 'include_tool_result_status', 'max_tokens', 'model_id', 'stop_sequences', 'streaming', 'temperature', 'top_p'].

See https://github.com/strands-agents/sdk-python/issues/815
  model = BedrockModel(


{'init_event_loop': True}
{'start': True}
{'start_event_loop': True}


ValidationException: An error occurred (ValidationException) when calling the ConverseStream operation: Invocation of model ID anthropic.claude-haiku-4-5-20251001-v1:0 with on-demand throughput isn’t supported. Retry your request with the ID or ARN of an inference profile that contains this model.

###  Tracking event loop lifecycle

This example illustrates the event loop lifecycle and how events relate to each other. It's useful for understanding the flow of execution in the Strands Agent:

Let's create some printing format code to better analyse the agent stream events. We will continue to use the same agent for it

In [4]:
# Async function that iterators over streamed agent events


async def process_streaming_response():
    agent_stream = agent.stream_async("What is the capital of France and what is 42+7?")
    async for event in agent_stream:
        # Track event loop lifecycle
        if event.get("init_event_loop", False):
            print("🔄 Event loop initialized")
        elif event.get("start_event_loop", False):
            print("▶️ Event loop cycle starting")
        elif event.get("start", False):
            print("📝 New cycle started")
        elif "message" in event:
            print(f"📬 New message created: {event['message']['role']}")
        elif event.get("force_stop", False):
            print(
                f"🛑 Event loop force-stopped: {event.get('force_stop_reason', 'unknown reason')}"
            )

        # Track tool usage
        if "current_tool_use" in event and event["current_tool_use"].get("name"):
            tool_name = event["current_tool_use"]["name"]
            print(f"🔧 Using tool: {tool_name}")

        # Show only a snippet of text to keep output clean
        if "data" in event:
            # Only show first 20 chars of each chunk for demo purposes
            data_snippet = event["data"][:20] + (
                "..." if len(event["data"]) > 20 else ""
            )
            print(f"📟 Text: {data_snippet}")

    return event["result"]


# Run the agent
asyncio.run(process_streaming_response())

🔄 Event loop initialized
📝 New cycle started
▶️ Event loop cycle starting
📟 Text: I
📟 Text:  can help you with
📟 Text:  the
📟 Text:  mathematical calcul...
📟 Text: , but I don
📟 Text: 't have a
📟 Text:  tool available
📟 Text:  to look
📟 Text:  up geographical
📟 Text:  information
📟 Text:  like capitals of
📟 Text:  countries. Let me
📟 Text:  calculate 
📟 Text: 42
📟 Text: +7 for
📟 Text:  you:
🔧 Using tool: calculator
🔧 Using tool: calculator
🔧 Using tool: calculator
🔧 Using tool: calculator
📬 New message created: assistant


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 42+7                │                                                                            │
│  │ Result    │ 49                  │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

📬 New message created: user
📝 New cycle started
📝 New cycle started
▶️ Event loop cycle starting
📟 Text: The result
📟 Text:  of 42 +
📟 Text:  7 is
📟 Text:  49.
📟 Text: 

As for the capital...
📟 Text:  France, while
📟 Text:  I know it is Paris,...
📟 Text:  don't have a specif...
📟 Text:  tool to verify or
📟 Text:  look up this inform...
📟 Text:  in the current
📟 Text:  environment.
📬 New message created: assistant


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "The result of 42 + 7 is 49.\n\nAs for the capital of France, while I know it is Paris, I don't have a specific tool to verify or look up this information in the current environment."}]}, metrics=EventLoopMetrics(cycle_count=4, tool_metrics={'calculator': ToolMetrics(tool={'toolUseId': 'tooluse_KJZb-ToGQFOM0KIasq7tdw', 'name': 'calculator', 'input': {'expression': '42+7'}}, call_count=2, success_count=2, error_count=0, total_time=0.012037038803100586)}, cycle_durations=[1.2419390678405762, 1.8845319747924805], traces=[<strands.telemetry.metrics.Trace object at 0x77af86530980>, <strands.telemetry.metrics.Trace object at 0x77af8651a2c0>, <strands.telemetry.metrics.Trace object at 0x77af86535150>, <strands.telemetry.metrics.Trace object at 0x77af86ce6d50>], accumulated_usage={'inputTokens': 8226, 'outputTokens': 234, 'totalTokens': 8460}, accumulated_metrics={'latencyMs': 7149}), state={})

### FastAPI Integration

You can also integrate your `stream_async` with FastAPI to create a streaming endpoint to your applications. For this, we will add a `weather_forecast` tool to our agent. The architecture update looks as following

<div style="text-align:left;">
    <img src="./images/architecture_2.png" width="65%" />
</div>

In [ ]:
# # Tool definition
from strands.models import BedrockModel
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"
model = BedrockModel(
    model_id=model_id,
)

@tool
def weather_forecast(city: str, days: int = 3) -> str:
    return f"Weather forecast for {city} for the next {days} days..."


# FastAPI app
app = FastAPI()


class PromptRequest(BaseModel):
    prompt: str


@app.post("/stream")
async def stream_response(request: PromptRequest):
    async def generate():
        agent = Agent( model = model, 
                      tools=[calculator, weather_forecast], callback_handler=None)
        try:
            async for event in agent.stream_async(request.prompt):
                if "data" in event:
                    yield event["data"]
        except Exception as e:
            yield f"Error: {str(e)}"

    return StreamingResponse(generate(), media_type="text/plain")


# Function to start server without blocking


async def start_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8001, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()


# Run server as background task
if "server_task" not in globals():
    server_task = asyncio.create_task(start_server())
    await asyncio.sleep(0.1)  # Give server time to start

print("✅ Server is running at http://0.0.0.0:8001")

INFO:     Started server process [338769]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


✅ Server is running at http://0.0.0.0:8001


INFO:     127.0.0.1:58480 - "POST /stream HTTP/1.1" 200 OK


#### Invoking the FastAPI agent
And we can now invoke the agent with a prompt 

In [6]:
async def fetch_stream():
    async with httpx.AsyncClient() as client:
        async with client.stream(
            "POST",
            "http://0.0.0.0:8001/stream",
            json={"prompt": "What is weather in NYC?"},
        ) as response:
            async for line in response.aiter_lines():
                if line.strip():  # Skip empty lines
                    print("Received:", line)


await fetch_stream()

Received: I'll check the current weather forecast for New York City.Based on the forecast, this is what we can expect for New York City over the next 3 days. (Note: I'm seeing simulated forecast data in this environment, but I've passed your request through to get the current forecast information.)


## Method 2 - Callback Handlers for streaming

Callback handlers are a powerful feature of the Strands Agents that allow you to intercept and process events as they happen during agent execution. This enables real-time monitoring, custom output formatting, and integration with external systems.



Callback handlers receive events in real-time as they occur during an agent's lifecycle:

- Text generation from the model
- Tool selection and execution
- Reasoning process
- Errors and completions


Let's now create a custom callback handler function that formats the event inputs to highlight tool usage and model output. To do so, we will again use the agent with a calculator tool only

<div style="text-align:left;">
    <img src="images/architecture.png" width="65%" />
</div>

In [8]:
def custom_callback_handler(**kwargs):
    # Process stream data
    if "data" in kwargs:
        print(f"MODEL OUTPUT: {kwargs['data']}")
    elif "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
        print(f"\nUSING TOOL: {kwargs['current_tool_use']['name']}")


# Create an agent with custom callback handler
agent = Agent(   
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",  # Optional: Specify the model ID
        tools=[calculator], 
        callback_handler=custom_callback_handler)

agent("Calculate 2+2")

MODEL OUTPUT: I
MODEL OUTPUT: 'll
MODEL OUTPUT:  help you calculate 2
MODEL OUTPUT: +2 using
MODEL OUTPUT:  the calculator
MODEL OUTPUT:  function
MODEL OUTPUT:  in evaluate
MODEL OUTPUT:  mode.

USING TOOL: calculator

USING TOOL: calculator

USING TOOL: calculator

USING TOOL: calculator

USING TOOL: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 2+2                 │                                                                            │
│  │ Result    │ 4                   │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

MODEL OUTPUT: The
MODEL OUTPUT:  result
MODEL OUTPUT:  is
MODEL OUTPUT:  4.


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The result is 4.'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'calculator': ToolMetrics(tool={'toolUseId': 'tooluse_vBmEv_o7R-GxSFY0eXR8mw', 'name': 'calculator', 'input': {'expression': '2+2'}}, call_count=1, success_count=1, error_count=0, total_time=0.004732847213745117)}, cycle_durations=[0.9648020267486572], traces=[<strands.telemetry.metrics.Trace object at 0x77af8537f9d0>, <strands.telemetry.metrics.Trace object at 0x77af7a7e5130>], accumulated_usage={'inputTokens': 3981, 'outputTokens': 82, 'totalTokens': 4063}, accumulated_metrics={'latencyMs': 2699}), state={})

### Congratulations!

In this notebook you learned how to stream your agents outputs using async iteractors and callback handlers. 